### Notebook for extracting STR dump for STR PRIORITISATION

#### Import Libraries

In [1]:
import sqlalchemy
import pyodbc
from urllib.parse import quote
import pandas as pd
import datetime
import numpy as np
import os
from tqdm import tqdm
from sqlalchemy.exc import OperationalError

In [2]:
# first file containing Batchid and Rptsernum

df1 = pd.read_excel('HIGH_VALUE_TRANSACTIONS_17-JUL-2023.xlsx')

In [3]:
# first file containing Batchid and Rptsernum

df2 = pd.read_excel('TF-17-JUL-2023.xlsx')

In [4]:
# Combining the two dataframes to get all STRs related batchid and rptsernum

complete_df = pd.concat([df1,df2])

In [5]:
complete_df['brpt'] = "'"+complete_df['BATCHID'].astype('str')+'|'+complete_df['RPTSERNUM'].astype('str')+"'"

In [6]:
complete_df = complete_df.drop_duplicates(subset='brpt')

In [7]:
complete_df.to_excel('batchids_str.xlsx', index = False)

In [8]:
# starting the sql engine

engine=sqlalchemy.create_engine('mssql+pyodbc://SAL_USR:%s@172.16.20.36:1433/HDM_DB?driver=ODBC+Driver+17+for+SQL+Server' % quote('Sal@read1'))

In [9]:
# Function to run the query


def query_run(query,engine_,path_to_file, file_name,mode = 'a'):
    '''
    **query**: ('str') Enter the query 
    **engine_** : ('str') the function creates a connection with postgres server 172.16.22.15:5432/postgres as default,
              other engines can be specified here as well
    **mode**: ('str') mode is append by default, can be changed to simply write as well
    **path_to_file**: ('str') path where the file to be saved
    **filename**: ('str') name of the file, defaults get saved as csv
    '''
    if mode == 'a':
        try:
            print('--Engine Running--')
            engine_ = engine_
            conn = engine_.connect()
            results = pd.read_sql(query,conn)
            conn.close()
            engine_.dispose()
            print('--Engine Closing--')
            print('Results_appending')
            file_name = f'{file_name}.csv'
            path = os.path.join(path_to_file,file_name)
            results.to_csv(path, index = False, mode = mode, header= not os.path.exists(path))
        except OperationalError as oe:
            time.sleep(3000)
            print('Reconnecting with Server')
            print('--Engine Running--')
            engine_ = engine_
            conn = engine_.connect()
            results = pd.read_sql(query,conn)
            conn.close()
            engine_.dispose()
            print('--Engine Closing--')
            print('Results_appending')
            filename = f'{file_name}.csv'
            path = os.path.join(path_to_file,file_name)
            results.to_csv(path, index = False, mode = mode, header= not os.path.exists(path))
    else:
        print('--Engine Running--')
        engine_ = engine_
        conn = engine_.connect()
        results = pd.read_sql(query,engine_)
        conn.close()
        engine_.dispose()
        print('--Engine Closing--')
        print('Results_appending')
        file_name = f'{file_name}.csv'
        path = os.path.join(path_to_file,file_name)
        results.to_csv(path, index = False, mode = mode)

In [10]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def remove_list_brackets(list_):
    str_no_list = str(list_)
    str_no_list = str_no_list.replace('[','(').replace(']',')')
    return str_no_list

In [12]:
import time

In [ ]:
for chunk in tqdm(list(chunks(complete_df['brpt'].unique().tolist(),2000))):
    chunk = remove_list_brackets(chunk)
    query = f"SELECT * FROM [HDM_DB].[FINCORE_BACKUP_CC].[ARFBRC] WHERE CONCAT(batchid,'|',rptsernum) in {chunk}"
    query = query.replace('"',"")
    engine_ = engine
    path_to_file = "D:\Str_prioritisation\data_dump"
    file_name = "arfbrc"
    query_run(query,engine_,path_to_file,file_name,mode = 'a')

  0%|                                                                                           | 0/39 [00:00<?, ?it/s]

--Engine Running--
